# Analyzing COVID RNA Sequences in Python

У цьому проекті ми збираємося завантажити та проаналізувати дані про послідовності РНК COVID у двох його основних варіантах: Дельта та Омікрон. РНК - це нуклеїнова кислота, присутня в будь-якій живій клітині, яка має один ланцюг, що складається з різних комбінацій чотирьох нуклеотидів: урацилу, цитозину, аденіну та гуаніну. РНК є «вихідним кодом» для COVID, який дозволяє вірусу проникати в клітину та розмножуватися.

In [ ]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
metadata = pd.read_csv("/kaggle/input/coronov/ncbi_datasets.csv")

In [ ]:
metadata.head()

Dataset має 16 полів та 847791 запис:

In [ ]:
metadata.shape

Для зручності аналізу краще змінити тип поля "Collection Date" з "object" на "datetime". Також назви полів слід писати через "_":

In [ ]:
metadata["Collection Date"] = pd.to_datetime(metadata["Collection Date"],format='mixed')

In [ ]:
metadata.columns = [col.lower().replace(" ", "_") for col in metadata.columns]

In [ ]:
metadata.columns

##  Exploratory Data Analysis

Мета аналізу - дати відповіді на наступні запитання:
- Коли була зібрана перша послідовність РНК на кожному континенті?
- Скільки послідовностей було зібрано на кожному континенті?
- Яка довжина найкоротшої та найдовшої послідовності?
- Скільки зразків збиралося за місяць? Чи є які-небудь тенденції?

In [ ]:
metadata['continent'] = metadata['geo_location'].str.replace(';.+','',regex = True)

In [ ]:
metadata

**Коли була зібрана перша послідовність РНК на кожному континенті?**

In [ ]:
metadata.groupby('continent').apply(lambda x: x.sort_values('collection_date').iloc[0])

**Скільки послідовностей було зібрано на кожному континенті?**

In [ ]:
metadata['continent'].value_counts()

**Яка довжина найкоротшої та найдовшої послідовності?**

In [ ]:
metadata['nucleotide_length'].describe()

In [ ]:
metadata[metadata['nucleotide_length'] == metadata['nucleotide_length'].min()]

In [ ]:
metadata[metadata['nucleotide_length'] == metadata['nucleotide_length'].max()]

**Скільки зразків збиралося за місяць? Чи є які-небудь тенденції?**

In [ ]:
sample_month = pd.Series (metadata['collection_date'].values.astype('<M8[M]'))

In [ ]:
sample_month.value_counts().sort_index().plot()

З січня 2020 року до січня 2021 року відбувалося різке збільшення кількості зразків. Після спаду з квітня по жовтень 2021 року, відбувся різкий вибух кількості нових зрізків(130000 в грудні 2021).

З графіку видно, що кількість зразків різко збільшується в зимовий період року.

## Identifying Sequences to Download

Тепер ми проаналізуємо самі не метадані, а наступні послідовності:

- Еталонна послідовність: перший геном COVID, який був повністю секвенований.

- Базова послідовність: наприклад, перша послідовність для Північної Америки.

- Одна з дельта-послідовностей (один із найпоширеніших варіантів COVID).

- Одна з послідовностей Omicron (ще один поширений варіант COVID).

**Перший геном COVID, який був повністю секвенований:**

In [ ]:
metadata[metadata['sequence_type'] == 'RefSeq']

**Перша послідовність для Північної Америки:**

In [ ]:
metadata[metadata['continent'] == 'North America'].sort_values('collection_date').iloc[0]

**Одна з дельта-послідовностей:**

In [ ]:
metadata[metadata['isolate_name'].str.contains("Delta").fillna(False)].iloc[0]

**Одна з послідовностей Omicron:**

In [ ]:
metadata[metadata['isolate_name'].str.contains("Omicron").fillna(False)]

In [ ]:
sequences = ['NC_045512.2','OM108163.1','OM095411.1','OL467832.1']
names = ['reference', 'base','delta','omicron']

In [ ]:
selected_sequences = metadata[metadata['nucleotide_accession'].isin(sequences)]

In [ ]:
selected_sequences